In [75]:
import matplotlib.pyplot as plt
import matplotlib.image as misc
import numpy as np
import network_module as nm


def init_train(src):
    global train
    def loadimage(src):
        print '  loading image:', src
        img=misc.imread(src)
        # loaded data is already normalized to [0,1]
        return img

    def settrain(img):
        train_data = np.dstack(np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]))).reshape(-1, 2)
        train_label = img.reshape(-1, 3)
        return (train_data, train_label)
    
    train = settrain(loadimage(src))
    return train


#########################
print 'Initializing trainset...'
init_train('google.jpg')
#########################


#########################
print 'Constructing network...'
# NETWORK DEFINITION
nn = nm.network(in_shape=train[0][0].shape, criterion='MSE')
nn.add_full(20)
nn.add_full(20)
nn.add_full(20)
nn.add_full(3)

#########################
print nn

result = []


def print_test():
    print ' --- Epoch: ', 1, ' error: ',\
          nn.output.get_crit(train[0][0:16], train[1][0:16]).mean()

def train_net():
    nn.SGD(train_policy=nn.fix_epoch,
           training_set=train,
           batch=16, rate=0.05, epoch_call_back=print_test, epoch=10)
    print 'Saving results to {}.res'.format(netname)
    print_csv('./results/{}.res'.format(netname), result)

    print 'Saving network snapshot to {}.net'.format(netname)
    nn.save_state('./nets/' + netname + '.net')



def loadnetwork(network_source):
    global nn
    nn = nm.load(network_source)

    
def imshow(im, cmap='Greys_r', interpol='None'):

    if len(im.shape) == 3:
        for i, x in enumerate(im, 1):
            plt.subplot(1, len(im), i)
            plt.imshow(x.squeeze(), cmap=cmap, interpolation=interpol)
            plt.gca().xaxis.set_major_locator(plt.NullLocator())
    if len(im.shape) == 4:
        for irow, xrow in enumerate(im, 0):
            for icol, x in enumerate(xrow, 1):
                print '\r  ', len(im), len(xrow), irow * len(xrow) + icol
                plt.subplot(len(im), len(xrow), irow * len(xrow) + icol)
                plt.imshow(x.squeeze(), cmap=cmap, interpolation=interpol)
                plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.show()
    return im.shape


def visualise_layer(lay_ind=4, top=9, iterations=1000):
    test = nn.grad_ascent(lay_ind, train[0], top, iterations)\
             .reshape((top,) + nn[lay_ind].shape + (28, 28))
    test = np.concatenate((test, test.mean(axis=0)[np.newaxis, :]), axis=0)
    return test


def max_act(lay_ind, top=9):
    return test[0][nn.max_act(lay_ind, test[0], top)].squeeze()



Initializing trainset...
  loading image: google.jpg
Constructing network...
Network ID: 139774980908880
Network layout:
------------------------------
	INPUT  (2,)
	   |
	   |
	  |1|
  fully connected  (20,)   ->   weights + bias: (20, 2) + (20,)
	   |
	   |
	  |2|
  fully connected  (20,)   ->   weights + bias: (20, 20) + (20,)
	   |
	   |
	  |3|
  fully connected  (20,)   ->   weights + bias: (20, 20) + (20,)
	   |
	   |
	  |4|
  fully connected  (3,)   ->   weights + bias: (3, 20) + (3,)
	   |
	   |
	  |5|
  	OUTPUT  (3,)   ->   CRITERION  (MSE)
------------------------------


In [76]:
train[0].shape

(952560, 2)

In [77]:
train[1].shape

(952560, 3)

In [78]:
train[0][-1]

array([ 755, 1259])

In [79]:
train[0][]

SyntaxError: invalid syntax (<ipython-input-79-9fde87cecfbc>, line 1)

In [13]:
nn[1].bias.shape

(1,)

In [80]:
img = mpimg.imread('google.jpg')
img.shape

(756, 1260, 3)

In [81]:
train_net()

   batch: 59535 of 59535  --- Epoch:  1  error:  9.38830448855e+91
   batch: 2996 of 59535

KeyboardInterrupt: 